<a href="https://colab.research.google.com/github/TheTrappist/teaching/blob/main/Biochem6761/AU24/01_Intro_AU24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Hi, and welcome to Biochem 6761!
# This is a comment. The computer won't do anything about it.
# Single-line comments are indicated by a pound '#' sign.

"""
Multi-line comments look
like this!
"""

# Let's have the computer actually do something.


# Remove the comments below to run the sample code

a = 5 # a is a variable. we can call it (almost) anything we want
"""
print(a)
print(a+5)

"""


In [ ]:
# Variables can (and often do) store sequences of data, like a column or a table
# in Excel. Let's see what that looks like.

# The following is an import statement. It loads additional functions stored in
# modules
import numpy as np

# Let's create a variable that stores a small linear array of data
x = np.linspace(1,5,5)
print(x)

# Let's pull the first element of this array
first_el = x[0] # note that the first element has the "index" of zero, not one
print(first_el)

# Now, let's pull the fourth element
"""
Remove comments and paste your code here!
"""


In [ ]:
# Excel starts having real trouble when your dataset reaches into the tens of
# thousands of datapoints. Let's see how Python does:
big_data = np.random.rand(1000,3)
print(big_data)

"""
Remove these comments and see how big you can go before you start running into
issues!
"""

In [ ]:
# Getting into some simple plotting: visualizing your data is supremely helpful!

# Import the standard plotting module
from matplotlib import pyplot as plt

small_data = np.random.rand(200,2)

x = small_data[:,0]
y = small_data[:,1]

plt.plot(x,y, marker='o', linestyle='')


In [ ]:
# Functions are a convenient way to group together several calculations

# This is the definition of a function that takes the inverse of a number
def take_inverse(x):
  inv = 1.0 / x
  return inv

#Here's how we can use it inside other code

test_num = 15
test_result = take_inverse(test_num)
print(test_result)


In [ ]:
"""
Your turn! In this cell, write a function that takes the average of two numbers.
Check to see if it works!
"""


In [ ]:
# Let's put some of the above together and do some plotting!

from matplotlib import pyplot as plt

x = np.linspace(0, 10, 100)

# This is a definition of a function
def exponent(x,a):
  y = np.power(x,a)
  return(y)

y = exponent(x,2)

plt.style.use("bmh")

# Let's make a simple plot
fig, ax = plt.subplots()
ax.plot(x,y)


In [ ]:
# Let's write a simple loop to see how this plot will change when we change
# the exponent

fig2, ax2 = plt.subplots()

for exp in np.linspace(0.4,3,8):
  ax2.plot(x, exponent(x,exp))


**This is a text cell. It won't trigger any code to run and can be used to annotate (describe) segments of code.**

# This is a cell storing text and equations in LaTeX.

The simple hyperbolic binding formula assumes that the concentration of receptor is very low:

$f_{bound}=\frac{[L]_{total}}{[L]_{total}+K_D}$


The fuller (and more accurate) quadratic form explicitly accounts for the concentration of protein/receptor:

$f_{bound}=\frac{([R]_{total}+[L]_{total}+K_D)-\sqrt{([R]_{total}+[L]_{total}+K_D)^2-4\times[R]_{total}\times{[L]_{total}}}}{2\times[R]_{total}}$

In both equations:

- $f_{bound}$ is the fraction of receptors that have a ligand bound
- $K_D$ is the dissociation constant for the ligand
- $[L]_{total}$ is the total concentration of the ligand in solution
- $[R]_{total}$ is the total concentration of protein (receptor) in solution


A fun paper for reference that blasts most published binding analyses:https://elifesciences.org/articles/57264

In [ ]:
# Rewrite the functions described in the previous cell in actual code

def f_bound_hyperbolic(L_tot,K_D):
  f_bound = 1 / (1 + K_D/L_tot)
  return f_bound

def f_bound_quadratic(L_tot, K_D, R_tot):
  # recast into standard quadratic equation for clarity
  a = R_tot
  b = -(R_tot+L_tot+K_D)
  c = L_tot
  f_bound = (-b - np.sqrt(b**2-4*a*c)) / (2*a)
  return f_bound

# Define a concentration range and parameters.
L_tot = np.linspace(0.1,500,500)
K_D = 50
R_tot = 300

fig, axs = plt.subplots(2,1)

f_hyper = f_bound_hyperbolic(L_tot,K_D)
f_quad = f_bound_quadratic(L_tot, K_D, R_tot)

axs[0].plot(L_tot, f_hyper)
axs[1].plot(L_tot, f_quad)
axs[1].set_xlabel('$L_{tot}$')


# Can you make the quadratic plot for a wide range of R_tot values?

"""
Replace this with your code!
"""

In [ ]:
# Now, let's practice downloading some real data

import pandas as pd # Pandas is a commonly used package in for working with data

binding_data = pd.read_csv('https://raw.githubusercontent.com/TheTrappist/teaching/main/Biochem6761/AU24/Sample_data/AU24_01_binding_Data_single.csv')

print(binding_data)

In [ ]:
# Let's plot these data and try to fit them!

# Load the nonlinear least squares fit function:
from scipy.optimize import curve_fit

x_vals = binding_data['ligand_nM']
y_vals = binding_data['frac_bound']

# plot the data
fig3, ax3 = plt.subplots()
ax3.plot(x_vals, y_vals, linestyle='', marker='o', color='g')

# Fit the data to a hyperbolic function
# this is the actual fitting step:
popt, pcov = curve_fit(f_bound_hyperbolic, x_vals, y_vals)


# Plot the fit:
x_axis_for_fit = np.linspace(0.1,200,200)
f_hyper_fit = f_bound_hyperbolic(x_axis_for_fit, popt[0])
ax3.plot(x_axis_for_fit, f_hyper_fit, color='b')
print("Kd from hyperbolic fit:", popt[0])

# Is it a good fit? Let's try the quadratic form

"""
# prepare function for fitting while keeping R_tot constant
R_tot = 50 # We need to define total receptor concentration
def quad_with_R_tot(L_tot, K_D):
  return(f_bound_quadratic(L_tot, K_D, R_tot))

# quadratic fitting step:
popt, pcov = curve_fit(quad_with_R_tot, x_vals, y_vals)
f_quad_fit = quad_with_R_tot(x_axis_for_fit, popt[0])
ax3.plot(x_axis_for_fit, f_quad_fit, color='r')

print("Kd from quadratic fit:", popt[0])
"""


In [ ]:
# Try it yourself with a larger dataset!

#Download the data from here:
'https://raw.githubusercontent.com/TheTrappist/teaching/main/Biochem6761/AU24/Sample_data/AU24_02_binding_Data_multi.csv'

# And try plotting and fitting it to quadratic and hyperbolic binding forms.